In [60]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [61]:
!unzip "dataset.zip"

Archive:  dataset.zip
replace dataset/test/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [78]:
import cv2
import os
import math
import numpy as np
from PIL import Image
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
from google.colab.patches import cv2_imshow

In [63]:
def get_path_list(root_path):
    '''
        To get a list of path directories from root path

        Parameters
        ----------
        root_path : str
            Location of root directory
        
        Returns
        -------
        list
            List containing the names of each person
    '''
    person_name = os.listdir(root_path)
    return person_name

In [64]:
def get_class_id(root_path, train_names):
    '''
        To get a list of train images and a list of image classes id

        Parameters
        ----------
        root_path : str
            Location of images root directory
        train_names : list
            List containing the names of the train sub-directories
        
        Returns
        -------
        list
            List containing all image in the train directories
        list
            List containing all image classes id
    '''
    img_list=[]
    class_list=[]
    for index,name in enumerate(train_names):
        folder_path = root_path +'/'+ name
        for image_name in os.listdir(folder_path):
            image_full_path = folder_path + '/' + image_name
            img_gray = cv2.imread(image_full_path)
            img_list.append(img_gray)
            class_list.append(index)
    return img_list,class_list

In [65]:
def detect_train_faces_and_filter(image_list, image_classes_list):
    '''
        To detect a face from given image list and filter it if the face on
        the given image is less than one

        Parameters
        ----------
        image_list : list
            List containing all loaded images
        image_classes_list : list
            List containing all image classes id
        
        Returns
        -------
        list
            List containing all filtered and cropped face images in grayscale
        list
            List containing all filtered image classes id
    '''
    filtered_list=[]
    filtered_classes_list=[]
    for index in range(len(image_classes_list)):
        img = image_list[index]
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        face_detected = face_cascade.detectMultiScale(img, 1.2,5)
        for face_rect in face_detected:
            x,y,w,h = face_rect
            face_img = img[y:y+h, x:x+w]
            # cv2.imshow('',face_img)
            # cv2.waitKey(0)
            filtered_list.append(face_img)
            filtered_classes_list.append(image_classes_list[index])
    return filtered_list,filtered_classes_list

In [66]:
def detect_test_faces_and_filter(image_list):
    '''
    To detect a face from given image list and filter it if the face on
    the given image is less than one

    Parameters
    ----------
    image_list : list
        List containing all loaded images
    
    Returns
    -------
    list
        List containing all filtered and cropped face images in grayscale
    list
        List containing all filtered faces location saved in rectangle
    '''

    test_img_loc=[]
    test_img_list=[]
    for img in image_list:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        face_detected = face_cascade.detectMultiScale(img,1.2,5)
        for face_rect in face_detected:
            x,y,w,h = face_rect
            face_img = img[y:y+h,x:x+w]
            test_img_list.append(face_img)
            test_img_loc.append(face_rect)
    return test_img_list,test_img_loc

In [67]:

def train(train_face_grays, image_classes_list):
    '''
        To create and train face recognizer object

        Parameters
        ----------
        train_face_grays : list
            List containing all filtered and cropped face images in grayscale
        image_classes_list : list
            List containing all filtered image classes id
        
        Returns
        -------
        object
            Recognizer object after being trained with cropped face images
    '''
    face_recog = cv2.face.LBPHFaceRecognizer_create()
    face_recog.train(train_face_grays,np.array(image_classes_list))
    return face_recog

In [68]:
def get_test_images_data(test_root_path):
    '''
        To load a list of test images from given path list

        Parameters
        ----------
        test_root_path : str
            Location of images root directory
        
        Returns
        -------
        list
            List containing all image in the test directories
    '''
    test_image=[]
    for image_name in os.listdir(test_root_path):
        image_full_path = test_root_path + '/' + image_name
        img = cv2.imread(image_full_path)
        test_image.append(img)
    return test_image

In [69]:
   
def predict(recognizer, test_faces_gray):
    '''
        To predict the test image with the recognizer

        Parameters
        ----------
        recognizer : object
            Recognizer object after being trained with cropped face images
        test_faces_gray : list
            List containing all filtered and cropped face images in grayscale

        Returns
        -------
        list
            List containing all prediction results from given test faces
    '''
    result=[]
    for image in test_faces_gray:
        res, confidence = recognizer.predict(image)
        result.append(res)
    return result

In [70]:

def draw_prediction_results(predict_results, test_image_list, test_faces_rects, train_names, size):
    '''
        To draw prediction results on the given test images and resize the image

        Parameters
        ----------
        predict_results : list
            List containing all prediction results from given test faces
        test_image_list : list
            List containing all loaded test images
        test_faces_rects : list
            List containing all filtered faces location saved in rectangle
        train_names : list
            List containing the names of the train sub-directories
        size : number
            Final size of each test image

        Returns
        -------
        list
            List containing all test images after being drawn with
            final result
    '''
    image_list=[]
    for index in range(len(test_image_list)):
        
        img = test_image_list[index]
        x,y,w,h = test_faces_rects[index]
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        
        
        img = cv2.resize(img,(size,size))
        text = train_names[predict_results[index]]
        cv2.putText(img,text,(0,20),cv2.FONT_HERSHEY_PLAIN, 2,(0,0,255),2)
        
        
        image_list.append(img)
    return image_list
     

In [81]:

def combine_and_show_result(image_list, size):
    '''
        To show the final image that already combine into one image

        Parameters
        ----------
        image_list : nparray
            Array containing image data
        size : number
            Final size of each test image
    '''
    res=[]
    for img in image_list:
        img = cv2.resize(img,(size,size))
        res.append(img)

    resimg=cv2.hconcat(res)

    cv2_imshow(resimg)
    

In [74]:
train_root_path = "dataset/train"
'''
    -------------------
    End of modifiable
    -------------------
'''

train_names = get_path_list(train_root_path)
train_image_list, image_classes_list = get_class_id(train_root_path, train_names)
print("Detect_train_faces")
train_face_grays, filtered_classes_list = detect_train_faces_and_filter(train_image_list, image_classes_list)
print("recognizer")


Detect_train_faces
recognizer


In [75]:
recognizer = train(train_face_grays, filtered_classes_list)

'''
    Please modify train_root_path value according to the location of
    your data train root directory

    -------------------
    Modifiable
    -------------------
'''
test_root_path = "dataset/test"
'''
    -------------------
    End of modifiable
    -------------------
'''


'\n    -------------------\n    End of modifiable\n    -------------------\n'

In [76]:
print("test_image_list")
test_image_list = get_test_images_data(test_root_path)
print("detect_test_faces")
test_faces_gray, test_faces_rects = detect_test_faces_and_filter(test_image_list)


test_image_list
detect_test_faces


In [ ]:
print("predict_result")
predict_results = predict(recognizer, test_faces_gray)
print("draw")
predicted_test_image_list = draw_prediction_results(predict_results, test_image_list, test_faces_rects, train_names, 200)
combine_and_show_result(predicted_test_image_list, 200)